### 최초 한번 실행 과거 데이터 추출용

In [1]:
from selenium import webdriver as wd
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import chromedriver_autoinstaller

In [2]:
from bs4 import BeautifulSoup as bs
import datetime, time

import pandas as pd
import numpy as np

import subprocess
import shutil
import pyperclip

import requests
import os, sys

In [3]:
try:
    shutil.rmtree(r"c:\chrometemp")  #쿠키 / 캐쉬파일 삭제. 정보를 지울 필요가 없으면 이부분은 그냥 skip
except FileNotFoundError:
    pass

In [4]:
try:
    subprocess.Popen(r'C:\Program Files\Google\Chrome\Application\chrome.exe --remote-debugging-port=9222 --user-data-dir="C:\chrometemp1"')  # 디버거 크롬 구동
except:
    subprocess.Popen(r'C:\Program Files (x86)\Google\Chrome\Application\chrome.exe --remote-debugging-port=9222 --user-data-dir="C:\chrometemp1"')  # 디버거 크롬 구동
# 위 두개의 위치중 한 곳에 chrome이 설치되어 있음.

In [5]:
option = Options()
option.add_experimental_option("debuggerAddress", "127.0.0.1:9222") # 포트 설정. 그냥 복붙하면 됨.

In [6]:
chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]
try:
    driver = wd.Chrome(f'./{chrome_ver}/chromedriver.exe', options=option)
except:
    chromedriver_autoinstaller.install(True)
    driver = wd.Chrome(f'./{chrome_ver}/chromedriver.exe', options=option)
driver.implicitly_wait(20)

In [8]:
# driver.set_window_position(-10000,0) # hide windows
main_url = 'https://kr.investing.com/'
driver.get(main_url)
time.sleep(1)

In [9]:
try: # pop up 창이 뜨는 것을 확인
    element = driver.find_element_by_id('PromoteSignUpPopUp')
    driver.find_element_by_css_selector('#PromoteSignUpPopUp > div.right > i').send_keys(Keys.ENTER)
    time.sleep(1)
    
except:
    pass

In [10]:
try: # 이미 로그인이 되어 있으면 skip
    driver.find_element_by_css_selector('#userAccount > div > a.login.bold').click() 
    time.sleep(1)
    
    id = 'kangeun@naver.com'
    pyperclip.copy(id) # control+c 기능 수행
    driver.find_element_by_id('loginFormUser_email').clear()
    driver.find_element_by_id('loginFormUser_email').send_keys(Keys.CONTROL + 'v')
    time.sleep(1)
    
    pw = os.environ['investing_pass']
    pyperclip.copy(pw) # control+c 기능 수행
    driver.find_element_by_id('loginForm_password').clear()
    driver.find_element_by_id('loginForm_password').send_keys(Keys.CONTROL + 'v')
    time.sleep(1)  
    
    driver.find_element_by_css_selector('#signup > a.newButton.orange').click()  
    time.sleep(2)
    
except:
    pass

In [11]:
file_csv = 'data_common.csv'
file_csv = 'data_common_test.csv'
df_org = pd.read_csv('../data/{}'.format(file_csv))
df_org['date'] = df_org['date'].apply(lambda x : datetime.datetime.strptime(x, "%Y-%m-%d"))
df_org.set_index('date', inplace=True)
df_org.replace(np.nan, '', inplace=True) 

In [12]:
spx_url = 'https://kr.investing.com/indices/us-spx-500-historical-data'
dji_url = 'https://kr.investing.com/indices/us-30-historical-data'
nas_url = 'https://kr.investing.com/indices/nasdaq-composite-historical-data'
kospi_url = 'https://kr.investing.com/indices/kospi-historical-data'
kosdaq_url = 'https://kr.investing.com/indices/kosdaq-historical-data'
sox_url = 'https://kr.investing.com/indices/phlx-semiconductor-historical-data'
# cpi_url = 'https://kr.investing.com/economic-calendar/cpi-733'
vix_url = 'https://kr.investing.com/indices/volatility-s-p-500-historical-data'

urls_major_index = [spx_url, dji_url, nas_url, kospi_url, kosdaq_url, sox_url, vix_url]


spx = ['date', 'spx', 'open', 'high', 'low', 'volume', 'spx_cr']
dji = ['date', 'dji', 'open', 'high', 'low', 'volume', 'dji_cr']
ixix = ['date', 'ixic', 'open', 'high', 'low', 'volume', 'ixic_cr']
kospi = ['date', 'kospi', 'open', 'high', 'low', 'volume', 'kospi_cr']
kosdaq = ['date', 'kosdaq', 'open', 'high', 'low', 'volume', 'kosdaq_cr']
sox = ['date', 'sox', 'open', 'high', 'low', 'volume', 'sox_cr']
# cpi = ['date', 'time', 'cpi', 'cpi_anticipated', 'cpi_previous', 'none']
# fng = ['date', 'n1', 'n2', 'n3', 'f_g_index']
vix = ['date', 'vix', 'open', 'high', 'low', 'volume', 'vix_cr']

col_names_major_index = [spx, dji, ixix, kospi, kosdaq, sox, vix ]

spx_s = ['date', 'spx', 'spx_cr']
dji_s = ['date', 'dji', 'dji_cr']
ixic_s = ['date', 'ixic', 'ixic_cr']
kospi_s = ['date', 'kospi', 'kospi_cr']
kosdaq_s = ['date', 'kosdaq', 'kosdaq_cr']
sox_s = ['date', 'sox', 'sox_cr']
# cpi_s = ['date', 'cpi', 'cpi_anticipated', 'cpi_previous']
# fng_s = ['date', 'f_g_index']
vix_s = ['date', 'vix', 'vix_cr']

col_names_s_major_index = [spx_s, dji_s, ixic_s, kospi_s, kosdaq_s, sox_s, vix_s]

In [13]:
us_bond_10yr_url = 'https://kr.investing.com/rates-bonds/u.s.-10-year-bond-yield-historical-data'
us_bond_2yr_url = 'https://kr.investing.com/rates-bonds/u.s.-2-year-bond-yield-historical-data'
us_bond_3mon_url = 'https://kr.investing.com/rates-bonds/u.s.-3-month-bond-yield-historical-data'
kor_bond_10yr_url = 'https://kr.investing.com/rates-bonds/south-korea-10-year-bond-yield-historical-data'
kor_bond_2yr_url = 'https://kr.investing.com/rates-bonds/south-korea-2-year-bond-yield-historical-data'

urls_bonds = [us_bond_10yr_url, us_bond_2yr_url, us_bond_3mon_url,
        kor_bond_10yr_url, kor_bond_2yr_url]

In [14]:
us_10yr = ['date', 'bond_usa_10', 'open', 'high', 'low', 'bond_usa_10_cr']
us_2yr = ['date', 'bond_usa_2', 'open', 'high', 'low', 'bond_usa_2_cr']
us_3mon = ['date', 'bond_usa_3m', 'open', 'high', 'low', 'bond_usa_3m_cr']
kor_10yr = ['date', 'bond_kor_10', 'open', 'high', 'low', 'bond_kor_10_cr']
kor_2yr = ['date', 'bond_kor_2', 'open', 'high', 'low','bond_kor_2_cr']

col_names_bonds = [us_10yr, us_2yr, us_3mon, kor_10yr, kor_2yr]

In [15]:
us_10yr_s = ['date', 'bond_usa_10', 'bond_usa_10_cr']
us_2yr_s = ['date', 'bond_usa_2', 'bond_usa_2_cr']
us_3mon_s = ['date', 'bond_usa_3m', 'bond_usa_3m_cr']
kor_10yr_s = ['date', 'bond_kor_10', 'bond_kor_10_cr']
kor_2yr_s = ['date', 'bond_kor_2', 'bond_kor_2_cr']

col_names_s_bonds = [us_10yr_s, us_2yr_s, us_3mon_s, kor_10yr_s, kor_2yr_s]

In [16]:
krw_rate_url = 'https://kr.investing.com/currencies/usd-krw-historical-data'

urls_currencies = [krw_rate_url]

In [17]:
krw_rate = ['date', 'krw', 'open', 'high', 'low', 'krw_cr']

col_names_currencies = [krw_rate]

In [18]:
krw_rate_s = ['date', 'krw', 'krw_cr']

col_names_s_currencies = [krw_rate_s]

In [19]:
nas_futures_url = 'https://kr.investing.com/indices/nq-100-futures-historical-data'
snp_futures_url = 'https://kr.investing.com/indices/us-spx-500-futures-historical-data'
dow_futures_url = 'https://kr.investing.com/indices/us-30-futures-historical-data'
wti_futures_url = 'https://kr.investing.com/commodities/crude-oil-historical-data'
dollar_index_url = 'https://kr.investing.com/currencies/us-dollar-index-historical-data'

urls_future = [snp_futures_url, dow_futures_url,  nas_futures_url, wti_futures_url, dollar_index_url ]

snp_future = ['date', 'spx_f', 'open', 'high', 'low', 'volume', 'spx_f_cr']
dji_future = ['date', 'dji_f', 'open', 'high', 'low', 'volume', 'dji_f_cr']
ixic_future = ['date', 'ixic_f', 'open', 'high', 'low', 'volume', 'ixic_f_cr']
wti_future = ['date', 'wti', 'open', 'high', 'low', 'volume', 'wti_cr']
dxy_future = ['date', 'dxy', 'open', 'high', 'low', 'volume', 'dxy_cr']

col_names_future = [snp_future,dji_future,ixic_future, wti_future, dxy_future]

snp_future_s = ['date', 'spx_f', 'spx_f_cr']
dji_future_s = ['date', 'dji_f', 'dji_f_cr']
ixic_future_s = ['date', 'ixic_f', 'ixic_f_cr']
wti_future_s = ['date', 'wti', 'wti_cr']
dxy_future_s = ['date', 'dxy', 'dxy_cr']

col_names_s_future  = [snp_future_s, dji_future_s, ixic_future_s, wti_future_s, dxy_future_s]

In [20]:
urls = urls_major_index + urls_bonds + urls_currencies + urls_future
col_names = col_names_major_index + col_names_bonds + col_names_currencies + col_names_future
col_names_s = col_names_s_major_index + col_names_s_bonds + col_names_s_currencies + col_names_s_future

In [21]:
def get_index_historical_data(start_date, end_date):
    df_append = None
    for sd, ed in zip(start_date, end_date):

        # driver.find_element(By.ID, 'widgetFieldDateRange').send_keys(Keys.ENTER)
        # Message: element not interactable error발생시 강제로 javascript로 click해주어야 함.
        element = driver.find_element(By.ID, 'widgetFieldDateRange')  
        driver.execute_script("arguments[0].click();", element)
        time.sleep(0.5)
        driver.find_element_by_id('startDate').clear()
        pyperclip.copy(sd) 
        driver.find_element_by_id('startDate').send_keys(Keys.CONTROL + 'v')
        time.sleep(1)
        driver.find_element_by_id('endDate').clear()
        pyperclip.copy(ed) 
        driver.find_element_by_id('endDate').send_keys(Keys.CONTROL + 'v')
        time.sleep(1)
#         driver.find_element_by_css_selector('#applyBtn').click()
#         driver.find_element_by_css_selector('#applyBtn').send_keys(Keys.ENTER)
        element = driver.find_element(By.ID, 'applyBtn')
        driver.execute_script("arguments[0].click();", element)
        time.sleep(3) # 시간을 줘야 함.
        df_t = pd.read_html(driver.page_source, 
                          attrs={"id": "curr_table"}, flavor=["lxml", "bs4"])[0]
        if df_append is None:
            df_append = df_t.copy()
            continue
        df_append = df_append.append(df_t, ignore_index=True)
        
    df_append.sort_values(by=[df_append.columns[0]], inplace=True)
    df_append.index = np.arange(0, len(df_append))
    return df_append

In [22]:
def make_datetime_series_df(df, start_date, end_date):
    # make dataframe with no empty date index with input df.
    # start_date =  '2021-01-01' str type.
    # df.index = datetime series
    # df.columns = ['a', 'b',,,,]
    date_range_ts = pd.date_range(start=start_date, end=end_date)
    df_empty = pd.DataFrame(columns = df.columns)
    df_empty.insert(0, 'date', date_range_ts)
    df_empty.replace(np.nan, '', inplace=True) # Nan을 빈 칸으로 대체
    df_empty.set_index('date', inplace=True)
    df_empty.update(df)
    return df_empty

In [23]:
# 검색하고자 하는 일정 기간 정의
start_date = \
     ['2020/01/01', '2020/03/01', '2020/05/01', '2020/07/01', '2020/09/01', '2020/11/01',
      '2021/01/01', '2021/03/01', '2021/05/01', '2021/07/01', '2021/09/01', '2021/11/01', 
      '2022/01/01', '2022/03/01', '2022/05/01', '2022/07/01']
end_date = \
     ['2020/02/29', '2020/04/30', '2020/06/30', '2020/08/31', '2020/10/31', '2020/12/31',
      '2021/02/28', '2021/04/30', '2021/06/30', '2021/08/31', '2021/10/31', '2021/12/31', 
      '2022/02/28', '2022/04/30', '2022/06/30', '2022/08/31']

In [24]:
# urls = [kor_bond_10yr_url, kor_bond_2yr_url]
# col_names = [kor_10yr, kor_2yr]
# col_names_s = [ kor_10yr_s, kor_2yr_s]

# start_date = ['2020/01/01', '2020/03/01']
# end_date = ['2020/02/29', '2020/04/30' ]

In [26]:
for i, url in enumerate(urls):
    driver.get(url)
    df = get_index_historical_data(start_date, end_date)

    print(url)
    df.columns = col_names[i]
    df['date'] = df['date'].apply(lambda x : datetime.datetime.strptime(x, "%Y년 %m월 %d일"))
    df.drop_duplicates(subset=['date'], keep='last', inplace=True) 
    # 중복항목 삭제. 필수 검토해야 함. 중복입력되는 경우 다수 발생
    df_select = df[col_names_s[i]]
    
    df_select.set_index('date', inplace=True)
    
    df_select = make_datetime_series_df(df_select, '2020-01-01', '2022-07-31')
    
    df_org.update(df_select)

https://kr.investing.com/rates-bonds/south-korea-10-year-bond-yield-historical-data
https://kr.investing.com/rates-bonds/south-korea-2-year-bond-yield-historical-data


In [ ]:
# df_org.reset_index(inplace=True)
# df_org.to_csv('data/data_common.csv', index=False)

#### get append cpi

In [27]:
cpi_url = 'https://kr.investing.com/economic-calendar/cpi-733'
driver.get(cpi_url)

In [28]:
for _ in range(5):
    driver.find_element_by_css_selector('#showMoreHistory733 > a').send_keys(Keys.ENTER)
    time.sleep(1)

In [29]:
df = pd.read_html(driver.page_source, flavor=["lxml", "bs4"])[0]
df.columns = ['date', 'time', 'cpi', 'cpi_anticipated', 'cpi_previous', 'none']
df['time'] = df['time'].apply(lambda x : datetime.datetime.strptime(x, "%H:%M").time())
df['date'] = df['date'].apply(lambda x : datetime.datetime.strptime(x[:13], "%Y년 %m월 %d일"))
df.drop_duplicates(subset=['date'], keep='last', inplace=True)

df_select = df[['date', 'cpi', 'cpi_anticipated', 'cpi_previous']]
df_select.set_index('date', inplace=True)

df_select = make_datetime_series_df(df_select, '2020-01-01', '2022-07-31')

df_org.update(df_select)

#### get append fear and greed

In [30]:
import pytz, json

In [31]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) \
           AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

In [32]:
BASE_URL = "https://production.dataviz.cnn.io/index/fearandgreed/graphdata"
START_DATE = '2021-01-01'  # no date before before 2020-08-01, error generated

In [33]:
def transform_timezone(x):
    dt = datetime.datetime.fromtimestamp(x / 1000, tz=pytz.utc)
#     tzone = pytz.timezone('US/Eastern')
    tzone = pytz.timezone('Asia/Seoul')
    loc_dt = dt.astimezone(tzone)
    return loc_dt.date()

In [34]:
r = requests.get("{}/{}".format(BASE_URL, START_DATE), headers=headers)
data = r.json()
fg_data = data['fear_and_greed_historical']['data']

In [35]:
df = pd.DataFrame(fg_data)
df.columns = ['date', 'f_g_index', 'rating']
# df.head()

In [36]:
df['date'] = df['date'].apply(lambda x: transform_timezone(x))
df['f_g_index'] = df['f_g_index'].astype('int')
df.drop_duplicates(subset=['date'], keep='last', inplace=True) # 중복항목 삭제. 필수 검토해야 함. 중복입력되는 경우 다수 발생
df_select = df[['date', 'f_g_index']]

df_select.set_index('date', inplace=True)

df_select = make_datetime_series_df(df_select, '2020-01-01', '2022-07-31')

df_org.update(df_select)

In [37]:
# df_org.reset_index(inplace=True)
# df_org.to_csv('data/data_common.csv', index=False)

#### get and append gold price

In [38]:
gold_url = 'https://www.usagold.com/daily-gold-price-history/'
driver.get(gold_url)

In [39]:
def get_gold_data():
    df_append = None
    for yr in ['2020', '2021', '2022']:

        driver.find_element(By.CSS_SELECTOR, '#goldpricemenu > button').send_keys(Keys.ENTER)
        time.sleep(1)
        driver.find_element(By.LINK_TEXT, yr).send_keys(Keys.ENTER)
        time.sleep(2)
        
        df = pd.read_html(driver.page_source, flavor=["lxml", "bs4"])
        df = df[0].drop(0) # delete empty first row
        df.columns = ['date', 'gold']
        df['date'] = df['date'].apply(lambda x : datetime.datetime.strptime(x, "%d %b %Y"))

        if df_append is None:
            df_append = df.copy()
            continue
        df_append = df_append.append(df, ignore_index=True)
        
    df_append.sort_values(by=[df_append.columns[0]], inplace=True)
    df_append.drop_duplicates(subset=['date'], inplace=True)
    df_append.index = np.arange(0, len(df_append))
    
    return df_append

In [40]:
df = get_gold_data()

In [41]:
# calculate change_rate
df_shift_p1 = df['gold'].shift(1)
df['gold_cr'] = (df['gold'] - df_shift_p1)/df_shift_p1
df.replace(np.nan, '', inplace=True)
# df.head()

In [42]:
df.set_index('date', inplace=True)

df = make_datetime_series_df(df, '2020-01-01', '2022-07-31')

df_org.update(df)

In [43]:
# df.reset_index(inplace=True)
# df.to_csv('data/data_gold.csv', index=False)

### fed 금리 get append

In [44]:
interest_url = 'https://www.investing.com/economic-calendar/interest-rate-decision-168/'
driver.get(interest_url)

In [45]:
for _ in range(6):
    driver.find_element_by_css_selector('#showMoreHistory168 > a').send_keys(Keys.ENTER)
    time.sleep(1)

In [46]:
df = pd.read_html(driver.page_source, attrs ={'id':'eventHistoryTable168'}, flavor=["lxml", "bs4"])[0]

In [47]:
df.columns = ['date', 'time', 'fed_rate', 'fed_rate_fore', 'fed_rate_prev', 'none']
df.replace(np.nan, '', inplace=True)

In [48]:
df['date'] = df['date'].apply(lambda x : datetime.datetime.strptime(x[:12], "%b %d, %Y"))
df['time'] = df['time'].apply(lambda x : datetime.datetime.strptime(x, "%H:%M").time())
df.drop_duplicates(subset=['date'], keep='last', inplace=True)

In [49]:
df_select = df[['date', 'fed_rate', 'fed_rate_fore', 'fed_rate_prev']]
df_select.set_index('date', inplace=True)

df_select = make_datetime_series_df(df_select, '2020-01-01', '2022-07-31')

df_org.update(df_select)

### 한국은행 금리 get append

In [50]:
kor_url = 'https://www.bok.or.kr/portal/singl/baseRate/list.do?dataSeCd=01&menuNo=200643'
driver.get(kor_url)

In [51]:
df = pd.read_html(kor_url, attrs = {'class': 'fixed'}, flavor=["lxml", "bs4"])[0]

In [52]:
df.columns=  ['release_yr', 'release_date', 'bok_rate']
df_date_temp = df['release_yr'].astype('str')+df['release_date']
df['date'] = df_date_temp.apply(lambda x : datetime.datetime.strptime(x, "%Y%m월 %d일"))
df.drop_duplicates(subset=['date'], keep='last', inplace=True)

In [53]:
df_select = df[['date', 'bok_rate']] # leave only valid columns
df_select.set_index('date', inplace=True)

df_select = make_datetime_series_df(df_select, '2020-01-01', '2022-07-31')

df_org.update(df_select)

### make csv file

In [55]:
df_org.reset_index(inplace=True)
df_org.to_csv('../data/{}'.format(file_csv), index=False)

In [56]:
driver.close()
driver.quit()